# Task 4: Portfolio Optimization and Risk Management

In this notebook, we will construct an optimized portfolio using three assets:
1. **TSLA (Forecasted Asset)**: Use the return forecast generated by the best-performing model (ARIMA).
2. **BND (Bond ETF)**: Use historical mean returns.
3. **SPY (S&P 500 ETF)**: Use historical mean returns.

We will use the Mean-Variance Optimization (MVO) approach to find the weights that maximize the Sharpe Ratio and minimize volatility.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# Add src to path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from models import ARIMAModel
from portfolio_optimizer import PortfolioOptimizer
from pypfopt import expected_returns

logging.basicConfig(level=logging.INFO)
warnings = __import__('warnings')
warnings.filterwarnings('ignore')

In [ ]:
# 1. Load Data
data = pd.read_csv('../data/processed/historical_data.csv', index_col=0, parse_dates=True, header=[0, 1])

# Extract Close Prices
tickers = ['TSLA', 'BND', 'SPY']
close_prices = pd.DataFrame()
for ticker in tickers:
    close_prices[ticker] = data[ticker]['Close']

print("Sample of Close Prices:")
print(close_prices.tail())

## 2. Generate Forecast for TSLA

We'll use the ARIMA model to forecast TSLA prices for the next 1 year (252 trading days).

In [ ]:
forecast_period = 252  # ~1 trading year
tsla_prices = close_prices['TSLA']

# Fit ARIMA on full TSLA data
arima_model = ARIMAModel()
arima_model.optimize_and_fit(tsla_prices)
tsla_forecast = arima_model.predict(n_periods=forecast_period)

# Calculate Annualized Expected Return for TSLA from forecast
current_price = tsla_prices.iloc[-1]
forecast_price_end = tsla_forecast.iloc[-1]
tsla_expected_return = (forecast_price_end - current_price) / current_price

print(f"TSLA Current Price: {current_price:.2f}")
print(f"TSLA Forecasted Price (1Y): {forecast_price_end:.2f}")
print(f"TSLA Expected Annual Return: {tsla_expected_return:.2%}")

## 3. Calculate Portfolio Metrics

We will calculate historical returns for BND and SPY, and use our forecasted return for TSLA.

In [ ]:
# Calculate historical annualized returns for all
mu_historical = expected_returns.mean_historical_return(close_prices)

# Override TSLA with forecasted return
mu_combined = mu_historical.copy()
mu_combined['TSLA'] = tsla_expected_return

print("Combined Expected Returns (Annualized):")
print(mu_combined)

## 4. Optimize Portfolio

We will run the optimizer to find the weights for both the Maximum Sharpe Ratio and Minimum Volatility.

In [ ]:
optimizer = PortfolioOptimizer(close_prices)
optimizer.calculate_metrics(mu_override=mu_combined)

# 1. Max Sharpe Ratio
max_sharpe_weights = optimizer.optimize_portfolio()

# 2. Min Volatility
min_vol_weights = optimizer.optimize_min_volatility()

print("\nMax Sharpe Ratio Weights:")
for ticker, weight in max_sharpe_weights.items():
    print(f"{ticker}: {weight:.2%}")

print("\nMin Volatility Weights:")
for ticker, weight in min_vol_weights.items():
    print(f"{ticker}: {weight:.2%}")

## 5. Summary and Comparison

We'll visualize the risk/return metrics for both strategies.

In [ ]:
# Pie chart comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

ax1.pie(max_sharpe_weights.values(), labels=max_sharpe_weights.keys(), autopct='%1.1f%%', startangle=140)
ax1.set_title("Max Sharpe Ratio Allocation")

ax2.pie(min_vol_weights.values(), labels=min_vol_weights.keys(), autopct='%1.1f%%', startangle=140)
ax2.set_title("Min Volatility Allocation")

plt.show()